In [1]:
library(shiny)
library(ggplot2)
reference= read.csv(file="/Users/niloofar/Downloads/phecode_definitions1.25.csv",sep=",")
comon=read.csv(file="/Users/niloofar/commoncomo.csv",sep=",")
vouge=c(
  "Random mental disorder. Ignored for now",
  "Abnormal findings on study of brain, nervous system",
  "Foreign body injury",
  "Complications of gastrostomy, colostomy and enterostomy",
  "Symptoms involving nervous and musculoskeletal systems",
  "Symptoms concerning nutrition, metabolism, and development",
  "Symptoms involving digestive system",
  "Symptoms of the muscles",
  "Other tests",
  "Effects of other external causes",
  "Other endocrine disorders",
  "Other symptoms",
  "Other conditions of brain",
  "Other diseases of lung",
  "Other disorders of metabolic, endocrine, immunity disorders",
  "Persistent mental disorders due to other conditions",
  "Other persistent mental disorders due to conditions classified elsewhere",
  "Other persistent mental disorders due to conditions classified elsewhere",
  "Other disorders of the nervous system",
  "Other nutritional deficiency",
  "Other ill-defined and unknown causes of morbidity and mortality",
  "Complications of surgical and medical procedures",
  "Nonspecific abnormal findings on radiological and other examination of skull and head",
  "Insect bite","Nonspecific abnormal results of other endocrine function study",
  "Burns","Calculus of ureter","Chemotherapy","Insulin pump user",
  "Nonspecific abnormal results of function study of brain and central nervous system",
  "Abnormal findings on study of brain and/or nervous system",
  "Other disorders of metabolism",
  "Other and unspecified disorders of the nervous system",
  "Other conditions of brain, NOS",
  "Late effects of cerebrovascular disease",
  "Other and unspecified disorders of the nervous system",
  "Other and unspecified congenital anomalies", "Ingrowing nail",
  "Transient mental disorders due to conditions classified elsewhere"
)


load_dat=function(t){
  table=read.csv(file=sprintf("phewasG%s.csv",t),sep=",")
  table=table[,-c(1)]
  table=table[table$A.OR.95..CI.!="(0-Inf)",]
  table=table[!table$Phenotype %in% vouge,]
  table$asdCount=gsub("[][()]", "",table$tPheno. )
  table$asdCount=sapply(table$asdCount,function(x) unlist(strsplit(x, "/"))[1])
  table$asdCount=as.numeric(table$asdCount)
  table=table[,c(12,2,3,6,9,13,4,7,5,8,10,11,1)]
  names(table)[1:6]=c("Phenotype","OR","CI","Pval","(case/Control)","case","GMOR","GMPval","GMCI")
  table$Pval=signif(table$Pval,3)
  table=merge(table,reference[,c(2,8)],by.x="Phenotype",by.y="phenotype")
  # table$GMPval=signif(table$GMPval,3)
  return(table[,c(1:6,14)])
}

G1=load_dat(1)
G2=load_dat(2)
G3=load_dat(3)
G4=load_dat(4)

####################
load("~/commonrefG1.RData")
load("~/commonrefG2.RData")
load("~/commonrefG3.RData")
load("~/commonrefG4.RData")

input1=data.frame(V1[[5]])
input2=data.frame(V2[[13]])
input3=data.frame(V3[[8]])
input4=data.frame(V4[[11]])


Input=function(i){
  return(get(sprintf("input%s",i)))
}


SubAll=function(input){
  L=lapply(input,Input)
  if(length(input)==1){return(L)}
  else{L=lapply(L,function(k) k$x) 
  return(Reduce(intersect,L))}
}

SubEx=function(input){
  a=c(1,2,3,4)
  b=a[!a%in% input]
  L=lapply(b,Input)
  L=lapply(L,function(k) k$x)
  L=Reduce(union,L)
  L=as.vector(L)
  tt=SubAll(input)
  tt=data.frame(tt)
  if (length(input)==1){return(tt[!tt$x %in% L,])}
  else{return(setdiff(tt,L))}
  
}



NumComo=function(input){
  d=sapply(get(sprintf("input%s",input))$x, function(x) strsplit(as.character(x), " \\* "))
  D=data.table::rbindlist(lapply(d, function(x) data.frame(x)))
  return(unique(as.character(D$x)))}

# SubAll=function(input){
#   L=lapply(input,Input)
#   return(Reduce(intersect,L))
# }
# 
# SubEx=function(input){
#   a=c(1,2,3,4)
#   b=a[!a%in% input]
#   L=lapply(b,Input)
#   L=Reduce(union,L)
#   tt=SubAll(input)
#   return(setdiff(tt,L))
#   
# }
# 

comoAll=function(V){
  L=lapply(V,NumComo)
  col=data.frame(Reduce(intersect,L))
  names(col)[1]="phenotype"
  col=merge(col,reference[,c(2,8)],by="phenotype")
  return(col)
}

comoEx=function(input){
  a=c(1,2,3,4)
  b=a[!a%in% input]
  L=lapply(b,NumComo)
  L=Reduce(union,L)
  tt=comoAll(input)
  col=data.frame(setdiff(tt,L))
  names(col)[1]="phenotype"
  col=merge(col,reference[,c(2,8)],by="phenotype")
  return(col)
}
variable=c(1,2,3,4)


#########################
ui = tagList(
    
    navbarPage(
      # theme = "cerulean",  # <--- To use a theme, uncomment this
      "Results",
      tabPanel("PheWAS",
               
               mainPanel(
                 titlePanel("Phewas for different Age groups"),
                 
                 sidebarLayout(
                   
                   # Sidebar panel for inputs ----
                   sidebarPanel(
                     
                     # Input: Choose dataset ----
                     selectInput("dataset", "Choose the Age Group:",
                                 choices = c("(0-2)", "(3-5)", "(6-11)","(12-18)","Common"))),
                   mainPanel(
                     
                     tableOutput("table")
                     
                   )),
                 
                 
                 fluidRow(
                   
                   
                   column(4,
                          selectInput("Pval",
                                      "Pvalue:",
                                      c("All","Bonferroni-Adjusted(0.01)","Bonferroni-Adjusted(0.05)","Bonferroni-Adjusted(0.1)"))),
                   
                   column(4,
                          selectInput("OR",
                                      "OR:",
                                      c("All",">2",">1.5","<1"))),
                   column(4,
                          selectInput("case",
                                      "  Total Number of Patients:",
                                      c("All",">10",">20",">50")))
                   
                 ),
                 
                 
                 fluidRow(
                   DT::dataTableOutput("table2")
                 ))
               
               
                 
      ),
      tabPanel("ASD Subtypes",
               
               sidebarLayout( 
                 sidebarPanel(
                   radioButtons("condition", "Select the Input:",
                                c("Subtype" ,
                                  "Comorbidity" )),
                   
                   
                   radioButtons("type", "Select the Type:",
                                c("Mutual" ,
                                  "MutualEx")),
                   
                   
                   
                   checkboxGroupInput("x", "Define the Age group:",
                                      c("(0-2)"=1,"(3-5)"=2,"(6-11)"=3,"(12-18)"=4))), 
                 
                 mainPanel (
                   DT::dataTableOutput("data")
                 ))
               
      )
      
               
    )
      
    )
  
  server = function(input, output) {
    
    datasetInput <- reactive({
      switch(input$dataset,
             "(0-2)" = G1,
             "(3-5)" = G2,
             "(6-11)" = G3,
             "(12-18)"= G4,
             "Common"= comon

      )})
    
    # Table of selected dataset ----
    output$table2 <- DT::renderDataTable(DT::datatable({
      data=datasetInput( )
    
      print(head(data))
      
      if (input$Pval == "Bonferroni-Adjusted(0.01)") {
        data <- data[data$Pval<=(0.01/nrow(data)),]
      }
      else if (input$Pval == "Bonferroni-Adjusted(0.1)") {
        data <- data[data$Pval<=(0.1/nrow(data)),]
      }
      else if (input$Pval == "Bonferroni-Adjusted(0.05)") {
        data <- data[data$Pval<=(0.05/nrow(data)),]
      }
      if (input$OR== ">2") {
        data <- data[data$OR>2,]
      }
      else if (input$OR == ">1.5") {
        data <- data[data$OR>1.5,]
      }
      else if (input$OR == "<1") {
        data <- data[data$OR<1,]
      }
      
      if (input$case== ">10") {
        data <- data[data$case>10 ,]
      }
      else if (input$case == ">20") {
        data <- data[data$case >20 ,]}
      else if (input$case==">50") {
        data <- data[data$case>50 ,]}
      # print("----------")
      # print(class(data))
      
      # print(str(data))
      data}))

    
  
    
  FuncType <- function() {
    if (input$condition=="Subtype"){
      switch(input$type,
             Mutual = SubAll(input$x),
             MutualEx=SubEx(input$x))}
    else if (input$condition=="Comorbidity"){
      switch(input$type,
             Mutual = comoAll(input$x),
             MutualEx=comoEx(input$x))}

  }

  datasetInput1 <- reactive(FuncType())
  output$data <- DT::renderDataTable(DT::datatable({
    data=data.frame(datasetInput1())
    print(head(data))
    data}))
   }

options(shiny.port = 8100)
shinyApp(ui = ui, server = server) 



Warning message in ensureInitialized():
“'.Random.seed[1]' is not a valid integer, so ignored”Warning message in names(table)[1:6] <- c("Phenotype", "OR", "CI", "Pval", "(case/Control)", :
“number of items to replace is not a multiple of replacement length”Warning message in names(table)[1:6] <- c("Phenotype", "OR", "CI", "Pval", "(case/Control)", :
“number of items to replace is not a multiple of replacement length”Warning message in names(table)[1:6] <- c("Phenotype", "OR", "CI", "Pval", "(case/Control)", :
“number of items to replace is not a multiple of replacement length”Warning message in names(table)[1:6] <- c("Phenotype", "OR", "CI", "Pval", "(case/Control)", :
“number of items to replace is not a multiple of replacement length”
Listening on http://127.0.0.1:8100
ERROR while rich displaying an object: Error in startServer(host, port, handlerManager$createHttpuvApp()): Failed to create server

Traceback:
1. FUN(X[[i]], ...)
2. tryCatch(withCallingHandlers({
 .     if (!mime %in%